In [ ]:
import torch
import time
import matplotlib.pyplot as plt
import non_local_boxes

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

# 1. Basic Gradient Descent

## 1.1. Constraint => Projection

As we maximize under constraints, we need the orthogonal projection onto the unit hypercube $[0,1]^{32}$:

In [ ]:
# def projected_wiring_basic(W):  # W is a 32xn tensor
#     W = torch.maximum(W, torch.zeros_like(W))  # it outputs the element-wise maximum
#     W = torch.minimum(W, torch.ones_like(W))   # similarly for minimum
#     return W

In [ ]:
# def projected_wiring_old(W):  # W is a 32xn tensor
#     W = torch.maximum(W, torch.zeros_like(W))  # it outputs the element-wise maximum
#     W = torch.minimum(W, torch.ones_like(W))   # similarly for minimum

#     # Then we verify the 4 equalities of (5)
#     for alpha in range(non_local_boxes.evaluate.nb_columns):
#         #1
#         if abs(W[0, alpha] - W[1, alpha]) <= abs(W[8, alpha] - W[9, alpha]):
#             W[0, alpha] = (W[0, alpha]+W[1, alpha])/2
#             W[1, alpha] = W[0, alpha]
#         else:
#             W[8, alpha] = (W[8, alpha]+W[9, alpha])/2
#             W[9, alpha] = W[8, alpha]
            
#         #2
#         if abs(W[2, alpha] - W[3, alpha]) <= abs(W[10, alpha] - W[11, alpha]):
#             W[2, alpha] = (W[2, alpha]+W[3, alpha])/2
#             W[3, alpha] = W[2, alpha]
#         else:
#             W[10, alpha] = (W[10, alpha]+W[11, alpha])/2
#             W[11, alpha] = W[10, alpha]
            
#         #3
#         if abs(W[4, alpha] - W[5, alpha]) <= abs(W[12, alpha] - W[13, alpha]):
#             W[4, alpha] = (W[4, alpha]+W[5, alpha])/2
#             W[5, alpha] = W[4, alpha]
#         else:
#             W[12, alpha] = (W[12, alpha]+W[13, alpha])/2
#             W[13, alpha] = W[12, alpha]
                
#         #4
#         if abs(W[6, alpha] - W[7, alpha]) <= abs(W[14, alpha] - W[15, alpha]):
#             W[6, alpha] = (W[6, alpha]+W[7, alpha])/2
#             W[7, alpha] = W[6, alpha]
#         else:
#             W[14, alpha] = (W[14, alpha]+W[15, alpha])/2
#             W[15, alpha] = W[14, alpha]

#     return W

In [ ]:
M1 = torch.zeros(32, 32)
for i in range(32):
    M1[i,i]=1
M1[0,0]=0.5
M1[0,1]=0.5
M1[1,0]=0.5
M1[1,1]=0.5

M2 = torch.zeros(32, 32)
for i in range(32):
    M2[i,i]=1
M2[8,8]=0.5
M2[8,9]=0.5
M2[9,8]=0.5
M2[9,9]=0.5

M3 = torch.zeros(32, 32)
for i in range(32):
    M3[i,i]=1
M3[2,2]=0.5
M3[2,3]=0.5
M3[3,2]=0.5
M3[3,3]=0.5

M4 = torch.zeros(32, 32)
for i in range(32):
    M4[i,i]=1
M4[10,10]=0.5
M4[10,11]=0.5
M4[11,10]=0.5
M4[11,11]=0.5

M5 = torch.zeros(32, 32)
for i in range(32):
    M5[i,i]=1
M5[4,4]=0.5
M5[4,5]=0.5
M5[5,4]=0.5
M5[5,5]=0.5

M6 = torch.zeros(32, 32)
for i in range(32):
    M6[i,i]=1
M6[12,12]=0.5
M6[12,13]=0.5
M6[13,12]=0.5
M6[13,13]=0.5

M7 = torch.zeros(32, 32)
for i in range(32):
    M7[i,i]=1
M7[6,6]=0.5
M7[6,7]=0.5
M7[7,6]=0.5
M7[7,7]=0.5

M8 = torch.zeros(32, 32)
for i in range(32):
    M8[i,i]=1
M8[14,14]=0.5
M8[14,15]=0.5
M8[15,14]=0.5
M8[15,15]=0.5


In [ ]:
def projected_wiring(W):  # W is a 32xn tensor
    W = torch.maximum(W, torch.zeros_like(W))  # it outputs the element-wise maximum
    W = torch.minimum(W, torch.ones_like(W))   # similarly for minimum

    T1 = (torch.abs(W[0,:]-W[1,:]) <= torch.abs(W[8, :] - W[9, :]))
    # If T1[i] is True: W[:,i] becomes M1 . W[:,i]
    # Otherwise: W[:,i] becomes M2 . W[:,i]
    W = T1*torch.tensordot(M1, W, dims=1) + torch.logical_not(T1)*torch.tensordot(M2, W, dims=1)
    
    T2 = (torch.abs(W[2,:]-W[3,:]) <= torch.abs(W[10, :] - W[11, :]))
    # If T2[i] is True: W[:,i] becomes M3 . W[:,i]
    # Otherwise: W[:,i] becomes M4 . W[:,i]
    W = T2*torch.tensordot(M3, W, dims=1) + torch.logical_not(T2)*torch.tensordot(M4, W, dims=1)

    T3 = (torch.abs(W[4,:]-W[5,:]) <= torch.abs(W[12, :] - W[13, :]))
    # If T3[i] is True: W[:,i] becomes M5 . W[:,i]
    # Otherwise: W[:,i] becomes M6 . W[:,i]
    W = T3*torch.tensordot(M5, W, dims=1) + torch.logical_not(T3)*torch.tensordot(M6, W, dims=1)

    T4 = (torch.abs(W[6,:]-W[7,:]) <= torch.abs(W[14, :] - W[15, :]))
    # If T4[i] is True: W[:,i] becomes M7 . W[:,i]
    # Otherwise: W[:,i] becomes M8 . W[:,i]
    W = T4*torch.tensordot(M7, W, dims=1) + torch.logical_not(T4)*torch.tensordot(M8, W, dims=1)

    return W

In [ ]:
# W = non_local_boxes.utils.random_wiring(non_local_boxes.evaluate.nb_columns).detach()
# projected_wiring(W) == projected_wiring_old(W)

In [ ]:
W = non_local_boxes.utils.W_BS09(non_local_boxes.evaluate.nb_columns).detach()
P = non_local_boxes.utils.SR
Q = non_local_boxes.utils.PR
float(non_local_boxes.evaluate.phi_flat(W, P, Q)[0])

## 1.2. Gradient Descent

In [ ]:
def gradient_descent(starting_W, P, Q, learning_rate = 2, nb_iterations = 20):
    external_grad = torch.ones(non_local_boxes.evaluate.nb_columns)
    W = starting_W
    for i in range(nb_iterations):
        non_local_boxes.evaluate.phi_flat(W, P, Q).backward(gradient=external_grad)
        W = projected_wiring(W + learning_rate*W.grad).detach()  # create a brand new tensor, forgeting the previous gradient
        W.requires_grad=True
    return W

In [ ]:
# Initialization
starting_W = non_local_boxes.utils.random_wiring(non_local_boxes.evaluate.nb_columns)
P = non_local_boxes.utils.SR
Q = non_local_boxes.utils.PR
learning_rate = 2
nb_iterations = 20


# Gradient Descent
print("(Iterating...)")
tic = time.time()
W = gradient_descent(starting_W, P, Q, learning_rate, nb_iterations)
toc = time.time()


# Result
list = non_local_boxes.evaluate.phi_flat(W, P, Q).detach().numpy()
print("")
print("Number of tested wirings: ", len(list))
print("--> Min:    ", min(list))
print("--> Av.:    ", sum(list)/len(list))
index, value = max(enumerate(list), key=lambda x: x[1])
best_wiring = W[:,index].detach().numpy()
print("--> Max:    ", value)
print("Duration: ", (toc-tic)*1e0, "s")
print("")
print("-----")
print("The best wiring is:\n", best_wiring)
print("-----")
non_local_boxes.utils.wiring_to_functions(best_wiring)

In [ ]:
def gradient_descent_timing(starting_W, P, Q, learning_rate = 2, nb_iterations = 20):
    external_grad = torch.ones(non_local_boxes.evaluate.nb_columns)
    W = starting_W
    Duration_evaluation = []
    Duration_gradient = []
    Duration_projection = []
    for i in range(nb_iterations):
        tic = time.time()
        evaluation = non_local_boxes.evaluate.phi_flat(W, P, Q)
        toc = time.time()
        Duration_evaluation.append(toc-tic)
        
        tic = time.time()
        evaluation.backward(gradient=external_grad)
        toc = time.time()
        Duration_gradient.append(toc-tic)

        tic = time.time()
        W = projected_wiring(W + learning_rate*W.grad).detach()  # create a brand new tensor, forgeting the previous gradient
        W.requires_grad=True
        toc = time.time()
        Duration_projection.append(toc-tic)

    Total = (torch.tensor(Duration_evaluation) + torch.tensor(Duration_gradient) + torch.tensor(Duration_projection)).numpy()
    plt.plot(Duration_evaluation, label = 'evaluation')
    plt.plot(Duration_gradient, label = 'gradient')
    plt.plot(Duration_projection, label = 'projection')
    plt.plot(Total, label = 'Total')
    return plt.legend()

In [ ]:
n = non_local_boxes.evaluate.nb_columns
W = non_local_boxes.utils.random_wiring(n)
P = non_local_boxes.utils.corNLB(0.5)

gradient_descent_timing(W, P, P, learning_rate=2, nb_iterations=50)

In [ ]:
#assert(False)

In [ ]:
# [0.24272715, 0.12160056, 0.9498826 , 0.02322189, 0.39412087,
#        0.39412087, 0.83709   , 0.52627677, 0.        , 0.        ,
#        1.        , 1.        , 0.        , 0.        , 1.        ,
#        1.        , 1.        , 0.        , 1.        , 0.        ,
#        1.        , 0.        , 1.        , 0.        , 1.        ,
#        0.        , 1.        , 0.        , 1.        , 0.        ,
#        1.        , 0.        ]

In [ ]:
# W = torch.t(torch.tensor([[0., 0., 1. , 0., 0.,
#        0., 1.   , 0.5 , 0.        , 0.        ,
#        1.        , 1.        , 0.        , 0.        , 1.        ,
#        1.        , 1.        , 0.        , 1.        , 0.        ,
#        1.        , 0.        , 1.        , 0.        , 1.        ,
#        0.        , 1.        , 0.        , 1.        , 0.        ,
#        1.        , 0.        ]]))

# P = non_local_boxes.utils.SR
# Q = non_local_boxes.utils.PR
# float(non_local_boxes.evaluate.phi_flat(W, P, Q)[0])

## 1.3 Test in the triangle PR-P0-P1

In [ ]:
X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()

triangle = plt.Polygon(X[:3,:], color="snow")
plt.gca().add_patch(triangle)

threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6
box_grid_size = 40
max_box_power = 1

PR = non_local_boxes.utils.PR
P_0 = non_local_boxes.utils.P_0
P_1 = non_local_boxes.utils.P_1
#W_BS09 = non_local_boxes.utils.W_BS09(non_local_boxes.evaluate.nb_columns).detach()
W_random = non_local_boxes.utils.random_wiring(non_local_boxes.evaluate.nb_columns)

for i in range(box_grid_size+1):
    for j in range(box_grid_size-i+1):
        alpha = i/box_grid_size
        beta = j/box_grid_size
        P = alpha*PR + beta*P_0 + (1-alpha-beta)*P_1   # P is a 4x4 matrix
        color_point = "orangered"

        Q=non_local_boxes.utils.matrix_to_tensor(torch.clone(P))  # Q is a 2x2x2x2 tensor 
        value = float(non_local_boxes.evaluate.h_flat(Q))

        if value > threshold:
            color_point = (0, 0.1, 0.1)
        else:
            # Given a box P, we look for the W maximizing P x_W P
            W = gradient_descent(W_random, P, P, learning_rate=2, nb_iterations=20)
            list = non_local_boxes.evaluate.phi_flat(W, P, P).detach().numpy()
            index, value = max(enumerate(list), key=lambda x: x[1])
            best_wiring = W[:,index].detach()
            best_wiring = torch.t(best_wiring.repeat(non_local_boxes.evaluate.nb_columns, 1))
            print(value)
            
            
            for k in range(max_box_power+1):
                if non_local_boxes.evaluate.h_flat(Q)[0] > threshold:
                    color_point = (0, 0.1*(1-k/max_box_power)+1*(k/max_box_power), 0.1*(1-k/max_box_power)+1*(k/max_box_power))
                    break
                #Q2=Q.copy()
                Q=non_local_boxes.evaluate.R(best_wiring, non_local_boxes.utils.tensor_to_matrix(Q), P)[:,:,:,:,0]

        plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)
                

plt.show()

## Gradient Descent of $W\mapsto P^{\boxtimes_W N}$: not very efficient

In [ ]:
assert(False)

In [ ]:
def gradient_descent_power(starting_W, P, N, learning_rate = 2, nb_iterations = 20):
    external_grad = torch.ones(non_local_boxes.evaluate.nb_columns)
    W = starting_W
    for i in range(nb_iterations):
        evaluate = non_local_boxes.evaluate.phi_power_recursive(W, P, N)
        evaluate.backward(gradient=external_grad)
        W = projected_wiring(W + learning_rate*W.grad).detach()  # create a brand new tensor, forgeting the previous gradient
        W.requires_grad=True
    return W

>À faire : il faut modifier phi_power et mettre des noms différents pour les Q, sinon l'algo ne retrouve pas les gradients...

In [ ]:
n = non_local_boxes.evaluate.nb_columns
W = non_local_boxes.utils.random_wiring(n)
P = non_local_boxes.utils.corNLB(0.5)
gradient_descent_power(W, P, 5, 2, 20)

# external_grad = torch.ones(non_local_boxes.evaluate.nb_columns)
# Q2 = non_local_boxes.evaluate.R(W, P, P)
# Q3 = torch.zeros(2, 2, 2, 2, n)
# for alpha in range(n):
#     Q3[:,:,:,:,alpha] = non_local_boxes.evaluate.R(W, non_local_boxes.utils.tensor_to_matrix(Q2[:,:,:,:,alpha]), P)[:,:,:,:,alpha]
# non_local_boxes.evaluate.phi_flat(W, non_local_boxes.utils.tensor_to_matrix(Q3[:,:,:,:,0]), P).backward(gradient=external_grad)
# W.grad



# external_grad = torch.ones(non_local_boxes.evaluate.nb_columns)
# Q = torch.clone(P)
# Q = non_local_boxes.utils.matrix_to_tensor(Q) # Q is a 2x2x2x2 tensor
# Q = non_local_boxes.evaluate.R(W, P, P) # Now on, Q is a 2x2x2x2xn tensor
# non_local_boxes.evaluate.h_flat(Q).backward(gradient=external_grad)
# W.grad


In [ ]:
n = non_local_boxes.evaluate.nb_columns
W = non_local_boxes.utils.W_BS09(n)
P = non_local_boxes.utils.corNLB(0.5)
print(non_local_boxes.evaluate.phi_power_recursive(W, P, 3))
Q = non_local_boxes.evaluate.R(W, P, P)[:,:,:,:,0]
Q = non_local_boxes.utils.tensor_to_matrix(Q)
print(non_local_boxes.evaluate.phi_flat(W, Q, P))

Gradient Descent for each power:

In [ ]:
X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()

triangle = plt.Polygon(X[:3,:], color="snow")
plt.gca().add_patch(triangle)

threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6
box_grid_size = 10
max_box_power = 2

PR = non_local_boxes.utils.PR
P_0 = non_local_boxes.utils.P_0
P_1 = non_local_boxes.utils.P_1
W_random = non_local_boxes.utils.random_wiring(non_local_boxes.evaluate.nb_columns)

for i in range(box_grid_size+1):
    for j in range(box_grid_size-i+1):
        alpha = i/box_grid_size
        beta = j/box_grid_size
        P = alpha*PR + beta*P_0 + (1-alpha-beta)*P_1   # P is a 4x4 matrix
        color_point = "orangered"

        
        #Q=torch.clone(P)
        #Q=non_local_boxes.utils.matrix_to_tensor(Q)  # Q is a 2x2x2x2 tensor 

        value = non_local_boxes.evaluate.h_flat(non_local_boxes.utils.matrix_to_tensor(P))[0]
        print(value)
        
        for k in range(1, max_box_power+1):

            if value > threshold:
                color_point = (0, 0.1*(1-k/max_box_power)+1*(k/max_box_power), 0.1*(1-k/max_box_power)+1*(k/max_box_power))
                break

            W = gradient_descent_power(W_random, P, k+1, learning_rate=2, nb_iterations=20)
            list = non_local_boxes.evaluate.phi_power_recursive(W, P, k+1).detach().numpy()
            index, value = max(enumerate(list), key=lambda x: x[1])  # find the best value in list
            #best_wiring = W[:,index].detach()
            #best_wiring = torch.t(best_wiring.repeat(non_local_boxes.evaluate.nb_columns, 1))
            print(value)
            #Q=non_local_boxes.evaluate.R(best_wiring, non_local_boxes.utils.tensor_to_matrix(Q), P)[:,:,:,:,0]

        plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)
                

plt.show()

> Coder à la main les puissances jusqu'à 10

# Puissances à la main

In [ ]:
def new_color(value, threshold, k, max_box_power_GD):
    if value > threshold:
        return (0, 0.1*(1-k/max_box_power_GD)+1*(k/max_box_power_GD), 0.1*(1-k/max_box_power_GD)+1*(k/max_box_power_GD))
    return "orangered"

In [ ]:
def next_Q(Q, W0, color_point, k, max_box_power_GD, learning_rate, nb_iterations, threshold): # Q is a 2x2x2x2 tensor
    if color_point == "orangered":
        W = gradient_descent(W0, non_local_boxes.utils.tensor_to_matrix(Q), P, learning_rate, nb_iterations)
        list = non_local_boxes.evaluate.phi_flat(W, non_local_boxes.utils.tensor_to_matrix(Q), P).detach().numpy()
        index, value = max(enumerate(list), key=lambda x: x[1])  # find the best value in list
        best_wiring = W[:,index].detach()
        best_wiring = torch.t(best_wiring.repeat(non_local_boxes.evaluate.nb_columns, 1))
        new_Q = non_local_boxes.evaluate.R(best_wiring, non_local_boxes.utils.tensor_to_matrix(Q), P)[:,:,:,:,0]
        color_point = new_color(value, threshold, k, max_box_power_GD)
        print(value)
        best_wiring.requires_grad = True
    else:
        new_Q = Q
        best_wiring = W0
    return new_Q, best_wiring, color_point

In [ ]:
def test_box_power(Q, P, W0, n, color_point, k, max_box_power_GD, max_box_power, learning_rate, nb_iterations, threshold): # Q is a 2x2x2x2 tensor
    for l in range(max_box_power):
        if color_point != "orangered":
            break
        Q=non_local_boxes.evaluate.R(W0, non_local_boxes.utils.tensor_to_matrix(Q), P)[:,:,:,:,0]
        value = float(non_local_boxes.evaluate.h_flat(Q))
        color_point = new_color(value, threshold, k, max_box_power_GD)

    return color_point


In [ ]:
X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()
triangle = plt.Polygon(X[:3,:], color="snow")
plt.gca().add_patch(triangle)

# constants
threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6
box_grid_size = 10
max_box_power = 50
max_box_power_GD = 5
learning_rate=2
nb_iterations=20

PR = non_local_boxes.utils.PR
P_0 = non_local_boxes.utils.P_0
P_1 = non_local_boxes.utils.P_1
n = non_local_boxes.evaluate.nb_columns
# W_random = non_local_boxes.utils.random_wiring(n)

for i in range(box_grid_size+1):
    for j in range(box_grid_size-i+1):
        alpha = i/box_grid_size
        beta = j/box_grid_size
        P = alpha*PR + beta*P_0 + (1-alpha-beta)*P_1   # P is a 4x4 matrix
        color_point = "orangered"

        
        value = float(non_local_boxes.evaluate.h_flat(non_local_boxes.utils.matrix_to_tensor(P)))
        color_point = new_color(value, threshold, 0, max_box_power_GD)

        Q=torch.clone(P)
        Q=non_local_boxes.utils.matrix_to_tensor(Q)  # Q is a 2x2x2x2 tensor 
        best_wiring = non_local_boxes.utils.random_wiring(n)

        for k in range(max_box_power_GD-1):
            Q, best_wiring, color_point = next_Q(Q, best_wiring, color_point, k+1, max_box_power_GD, learning_rate, nb_iterations, threshold)
            color_point = test_box_power(Q, P, best_wiring, n, color_point, k+1, max_box_power_GD, max_box_power, learning_rate, nb_iterations, threshold)

        
        # Q2, best_wiring, color_point = next_Q(non_local_boxes.utils.matrix_to_tensor(P), W_random, color_point)
        # Q3, best_wiring, color_point = next_Q(Q2, best_wiring, color_point)
        # Q4, best_wiring, color_point = next_Q(Q3, best_wiring, color_point)
        # Q5, best_wiring, color_point = next_Q(Q4, best_wiring, color_point)

        plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)
                

plt.show()

# Another triangle: $PR - P_0 - P_L^{1011}$

In [ ]:
X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()

triangle = plt.Polygon(X[:3,:], color="snow")
plt.gca().add_patch(triangle)

threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6
box_grid_size = 40
max_box_power = 50

PR = non_local_boxes.utils.PR
P_0 = non_local_boxes.utils.P_0
P_1 = non_local_boxes.utils.P_L(1,0,1,1)
#W_BS09 = non_local_boxes.utils.W_BS09(non_local_boxes.evaluate.nb_columns).detach()
W_random = non_local_boxes.utils.random_wiring(non_local_boxes.evaluate.nb_columns)

for i in range(box_grid_size+1):
    for j in range(box_grid_size-i+1):
        alpha = i/box_grid_size
        beta = j/box_grid_size
        P = alpha*PR + beta*P_0 + (1-alpha-beta)*P_1   # P is a 4x4 matrix
        color_point = "orangered"

        # Given a box P, we look for the W maximizing P x_W P
        W = gradient_descent(W_random, P, P, learning_rate=2, nb_iterations=20)
        list = non_local_boxes.evaluate.phi_flat(W, P, P).detach().numpy()
        index, value = max(enumerate(list), key=lambda x: x[1])
        best_wiring = W[:,index].detach()
        best_wiring = torch.t(best_wiring.repeat(non_local_boxes.evaluate.nb_columns, 1))
        print(value)
        
        Q=torch.clone(P)
        Q=non_local_boxes.utils.matrix_to_tensor(Q)  # Q is a 2x2x2x2 tensor 
        for k in range(max_box_power+1):
            if non_local_boxes.evaluate.h_flat(Q)[0] > threshold:
                color_point = (0, 0.1*(1-k/max_box_power)+1*(k/max_box_power), 0.1*(1-k/max_box_power)+1*(k/max_box_power))
                break
            #Q2=Q.copy()
            Q=non_local_boxes.evaluate.R(best_wiring, non_local_boxes.utils.tensor_to_matrix(Q), P)[:,:,:,:,0]

        plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)
                

plt.show()

# Blue cones

In [ ]:
X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()
triangle = plt.Polygon(X[:3,:], color="snow")
plt.gca().add_patch(triangle)

box_grid_size = 40
n = non_local_boxes.evaluate.nb_columns
W_random = non_local_boxes.utils.random_wiring(n)
alpha, beta = 0.2, 0.6
x0, y0 = X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta)
f = lambda x : X[1,1]*(x-x0)/(X[1,0]-x0) + y0*(x-X[1,0])/(x0-X[1,0])  # Lagrange interpolation
g = lambda x : X[2,1]*(x-x0)/(X[2,0]-x0) + y0*(x-X[2,0])/(x0-X[2,0])  # Lagrange interpolation

for i in range(box_grid_size+1):
    for j in range(i+1):
        alpha, beta = 1-i/box_grid_size, j/box_grid_size
        x, y = X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta)
        
        color_point = "orangered"
        if y < f(x) and y > g(x) :
            color_point="blue"
         
        if i == 0 and j == 0:
            color_point="green"
        plt.plot(x, y, 'o', markersize=3, color=color_point)
plt.show()

In [ ]:
def is_in_a_blue_cone(x, y, list, X): # It tests if Box located at (x, y) is in the cone of a blue box
    # list is a list of coordinates (x0, y0) such that h(P)<0.91 but P is blue
    for point in list:
        x0, y0 = point
        f = lambda x : X[1,1]*(x-x0)/(X[1,0]-x0) + y0*(x-X[1,0])/(x0-X[1,0])  # Lagrange interpolation
        g = lambda x : X[2,1]*(x-x0)/(X[2,0]-x0) + y0*(x-X[2,0])/(x0-X[2,0])  # Lagrange interpolation
        if y <= f(x) and y >= g(x):
            return True
    return False

In [ ]:
X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()
list = [(3.5, 2.5), (3.3, 3.8)]
is_in_a_blue_cone(3.3, 2.4, list, X)

In [ ]:
X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()
triangle = plt.Polygon(X[:3,:], color="snow")
plt.gca().add_patch(triangle)

box_grid_size = 40
list = [(3., 2.5), (3.3, 3.8)]
n = non_local_boxes.evaluate.nb_columns
W_random = non_local_boxes.utils.random_wiring(n)

for i in range(box_grid_size+1):
    for j in range(i+1):
        alpha, beta = 1-i/box_grid_size, j/box_grid_size
        x, y = X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta)
        
        color_point = "orangered"
        if is_in_a_blue_cone(x,y,list,X) :
            color_point="blue"
         
        if i == 0 and j == 0:
            color_point="green"
        plt.plot(x, y, 'o', markersize=3, color=color_point)
plt.show()

# In the triangle PR-P1-PRprime

In [ ]:
X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()

triangle = plt.Polygon(X[:3,:], color="snow")
plt.gca().add_patch(triangle)

threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6
box_grid_size = 40
max_box_power = 50

PR = non_local_boxes.utils.PR
#P_0 = non_local_boxes.utils.P_0
P_1 = non_local_boxes.utils.P_L(1,0,1,1)
PRprime = (non_local_boxes.utils.P_NL(1,1,1)+PR)/2
#W_BS09 = non_local_boxes.utils.W_BS09(non_local_boxes.evaluate.nb_columns).detach()
W_random = non_local_boxes.utils.random_wiring(non_local_boxes.evaluate.nb_columns)

for i in range(box_grid_size+1):
    for j in range(box_grid_size-i+1):
        alpha = i/box_grid_size
        beta = j/box_grid_size
        P = alpha*PR + beta*P_1 + (1-alpha-beta)*PRprime   # P is a 4x4 matrix
        color_point = "orangered"

        # Given a box P, we look for the W maximizing P x_W P
        W = gradient_descent(W_random, P, P, learning_rate=2, nb_iterations=20)
        list = non_local_boxes.evaluate.phi_flat(W, P, P).detach().numpy()
        index, value = max(enumerate(list), key=lambda x: x[1])
        best_wiring = W[:,index].detach()
        best_wiring = torch.t(best_wiring.repeat(non_local_boxes.evaluate.nb_columns, 1))
        print(value)
        
        Q=torch.clone(P)
        Q=non_local_boxes.utils.matrix_to_tensor(Q)  # Q is a 2x2x2x2 tensor 
        for k in range(max_box_power+1):
            if non_local_boxes.evaluate.h_flat(Q)[0] > threshold:
                color_point = (0, 0.1*(1-k/max_box_power)+1*(k/max_box_power), 0.1*(1-k/max_box_power)+1*(k/max_box_power))
                break
            #Q2=Q.copy()
            Q=non_local_boxes.evaluate.R(best_wiring, non_local_boxes.utils.tensor_to_matrix(Q), P)[:,:,:,:,0]

        plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)
                

plt.show()

# In the triangle PR-P0-PL0010

In [ ]:
X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()

triangle = plt.Polygon(X[:3,:], color="snow")
plt.gca().add_patch(triangle)

threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6
box_grid_size = 40
max_box_power = 50

PR = non_local_boxes.utils.PR
P_0 = non_local_boxes.utils.P_0
P_1 = non_local_boxes.utils.P_L(1,0,1,1)
Box = non_local_boxes.utils.P_L(0,0,1,0)
#W_BS09 = non_local_boxes.utils.W_BS09(non_local_boxes.evaluate.nb_columns).detach()
W_random = non_local_boxes.utils.random_wiring(non_local_boxes.evaluate.nb_columns)

for i in range(box_grid_size+1):
    for j in range(box_grid_size-i+1):
        alpha = i/box_grid_size
        beta = j/box_grid_size
        P = alpha*PR + beta*P_0 + (1-alpha-beta)*Box   # P is a 4x4 matrix
        color_point = "orangered"

        # Given a box P, we look for the W maximizing P x_W P
        W = gradient_descent(W_random, P, P, learning_rate=2, nb_iterations=20)
        list = non_local_boxes.evaluate.phi_flat(W, P, P).detach().numpy()
        index, value = max(enumerate(list), key=lambda x: x[1])
        best_wiring = W[:,index].detach()
        best_wiring = torch.t(best_wiring.repeat(non_local_boxes.evaluate.nb_columns, 1))
        print(value)
        
        Q=torch.clone(P)
        Q=non_local_boxes.utils.matrix_to_tensor(Q)  # Q is a 2x2x2x2 tensor 
        for k in range(max_box_power+1):
            if non_local_boxes.evaluate.h_flat(Q)[0] > threshold:
                color_point = (0, 0.1*(1-k/max_box_power)+1*(k/max_box_power), 0.1*(1-k/max_box_power)+1*(k/max_box_power))
                break
            #Q2=Q.copy()
            Q=non_local_boxes.evaluate.R(best_wiring, non_local_boxes.utils.tensor_to_matrix(Q), P)[:,:,:,:,0]

        plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)
                

plt.show()

# In the triangle PR-P1-PL0010

In [ ]:
X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()

triangle = plt.Polygon(X[:3,:], color="snow")
plt.gca().add_patch(triangle)

threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6
box_grid_size = 40
max_box_power = 50

PR = non_local_boxes.utils.PR
P_0 = non_local_boxes.utils.P_0
P_1 = non_local_boxes.utils.P_L(1,0,1,1)
Box = non_local_boxes.utils.P_L(0,0,1,0)
#W_BS09 = non_local_boxes.utils.W_BS09(non_local_boxes.evaluate.nb_columns).detach()
W_random = non_local_boxes.utils.random_wiring(non_local_boxes.evaluate.nb_columns)

for i in range(box_grid_size+1):
    for j in range(box_grid_size-i+1):
        alpha = i/box_grid_size
        beta = j/box_grid_size
        P = alpha*PR + beta*P_1 + (1-alpha-beta)*Box   # P is a 4x4 matrix
        color_point = "orangered"

        # Given a box P, we look for the W maximizing P x_W P
        W = gradient_descent(W_random, P, P, learning_rate=2, nb_iterations=20)
        list = non_local_boxes.evaluate.phi_flat(W, P, P).detach().numpy()
        index, value = max(enumerate(list), key=lambda x: x[1])
        best_wiring = W[:,index].detach()
        best_wiring = torch.t(best_wiring.repeat(non_local_boxes.evaluate.nb_columns, 1))
        print(value)
        
        Q=torch.clone(P)
        Q=non_local_boxes.utils.matrix_to_tensor(Q)  # Q is a 2x2x2x2 tensor 
        for k in range(max_box_power+1):
            if non_local_boxes.evaluate.h_flat(Q)[0] > threshold:
                color_point = (0, 0.1*(1-k/max_box_power)+1*(k/max_box_power), 0.1*(1-k/max_box_power)+1*(k/max_box_power))
                break
            #Q2=Q.copy()
            Q=non_local_boxes.evaluate.R(best_wiring, non_local_boxes.utils.tensor_to_matrix(Q), P)[:,:,:,:,0]

        plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)
                

plt.show()

# In the segment PR-PL0010

In [ ]:
X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()

triangle = plt.Polygon(X[:3,:], color="snow")
plt.gca().add_patch(triangle)

threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6
box_grid_size = 60
max_box_power = 2

PR = non_local_boxes.utils.PR
P_0 = non_local_boxes.utils.P_0
P_1 = non_local_boxes.utils.P_L(1,0,1,1)
Box = non_local_boxes.utils.P_L(0,0,1,0)
#W_BS09 = non_local_boxes.utils.W_BS09(non_local_boxes.evaluate.nb_columns).detach()
W_random = non_local_boxes.utils.random_wiring(non_local_boxes.evaluate.nb_columns)

for i in range(box_grid_size+1):
    #for j in range(box_grid_size-i+1):
        alpha = i/box_grid_size
        #beta = j/box_grid_size
        P = alpha*PR + (1-alpha)*Box   # P is a 4x4 matrix
        color_point = "orangered"

        # Given a box P, we look for the W maximizing P x_W P
        W = gradient_descent(W_random, P, P, learning_rate=2, nb_iterations=20)
        list = non_local_boxes.evaluate.phi_flat(W, P, P).detach().numpy()
        index, value = max(enumerate(list), key=lambda x: x[1])
        best_wiring = W[:,index].detach()
        best_wiring = torch.t(best_wiring.repeat(non_local_boxes.evaluate.nb_columns, 1))
        print(value)
        
        Q=torch.clone(P)
        Q=non_local_boxes.utils.matrix_to_tensor(Q)  # Q is a 2x2x2x2 tensor 
        for k in range(max_box_power+1):
            if non_local_boxes.evaluate.h_flat(Q)[0] > threshold:
                color_point = (0, 0.1*(1-k/max_box_power)+1*(k/max_box_power), 0.1*(1-k/max_box_power)+1*(k/max_box_power))
                break
            #Q2=Q.copy()
            Q=non_local_boxes.evaluate.R(best_wiring, non_local_boxes.utils.tensor_to_matrix(Q), P)[:,:,:,:,0]

        plt.plot(X[0,0]*alpha + X[2,0]*(1-alpha), X[0,1]*alpha + X[2,1]*(1-alpha), 'o', markersize=3, color=color_point)
                

plt.show()